In [14]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd 
import time

In [2]:
driver = webdriver.Chrome()
driver.get('https://winmart.vn/info/danh-sach-cua-hang')

# click để mở dropdown
input_box = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "province"))
)
input_box.click()

# chờ danh sách load
time.sleep(1)  # hoặc chờ tối ưu hơn như bên dưới

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "ul[role='listbox']"))
)

# lấy các item
items = driver.find_elements(By.CSS_SELECTOR, "li[role='option']")
province_list = [item.text.strip() for item in items]

driver.quit()

# lưu CSV
df = pd.DataFrame({"province": province_list})
df.to_csv("store_info/province_list.csv", index=False, encoding="utf-8")

print("✅ Done → province_list.csv")
print(province_list)

✅ Done → province_list.csv
['TP. Hà Nội', 'TP. Hồ Chí Minh', 'TP. Đà Nẵng', 'T. An Giang', 'T. Bắc Giang', 'T. Bắc Kạn', 'T. Bạc Liêu', 'T. Bắc Ninh', 'T. Bà Rịa - Vũng Tàu', 'T. Bến Tre', 'T. Bình Định', 'T. Bình Dương', 'T. Bình Thuận', 'T. Cà Mau', 'T. Đắk Lắk', 'T. Đồng Nai', 'T. Đồng Tháp', 'T. Gia Lai', 'T. Hải Dương', 'T. Hà Nam', 'T. Hà Tĩnh', 'T. Hậu Giang', 'T. Hòa Bình', 'T. Hưng Yên', 'T. Khánh Hòa', 'T. Kiên Giang', 'T. Kon Tum', 'T. Lai Châu', 'T. Lâm Đồng', 'T. Lạng Sơn', 'T. Long An', 'T. Nam Định', 'T. Nghệ An', 'T. Ninh Bình', 'T. Ninh Thuận', 'TP. Cần Thơ', 'TP. Hải Phòng', 'T. Phú Thọ', 'T. Phú Yên', 'T. Quảng Bình', 'T. Quảng Nam', 'T. Quảng Ngãi', 'T. Quảng Ninh', 'T. Quảng Trị', 'T. Sóc Trăng', 'T. Sơn La', 'T. Tây Ninh', 'T. Thái Bình', 'T. Thái Nguyên', 'T. Thanh Hóa', 'T. Thừa Thiên - Huế', 'T. Tiền Giang', 'T. Trà Vinh', 'T. Tuyên Quang', 'T. Vĩnh Long', 'T. Vĩnh Phúc', 'T. Yên Bái']


In [3]:
pip install unicodedata2

   ---------------------------------------- 0.0/479.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/479.6 kB ? eta -:--:--
    --------------------------------------- 10.2/479.6 kB ? eta -:--:--
   --- ----------------------------------- 41.0/479.6 kB 487.6 kB/s eta 0:00:01
   -------- ----------------------------- 112.6/479.6 kB 930.9 kB/s eta 0:00:01
   ------------------ --------------------- 225.3/479.6 kB 1.5 MB/s eta 0:00:01
   ------------------------------------ --- 440.3/479.6 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 479.6/479.6 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import unicodedata
import re

SPECIAL_CASES = {
    "T. Thừa Thiên - Huế": "tinh-thua-thien-hue",
    "T. Bà Rịa - Vũng Tàu": "tinh-ba-ria-vung-tau"
}

def vietnamese_to_slug(text):
    # Ưu tiên ngoại lệ
    if text in SPECIAL_CASES:
        return SPECIAL_CASES[text]
    

    text = text.lower().strip()

    # Thay thế đ/Đ
    text = text.replace("đ", "d").replace("Đ", "d")

    # Normalize -> remove accent marks
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')

    # Chuyển TP. / T. sang từ đầy đủ
    text = (text.replace("tp.", "thanh pho")
                .replace("tp ", "thanh pho ")
                .replace("t.", "tinh")
                .replace("t ", "tinh ")
                .replace("tinh", "tinh"))

    # Loại ký tự không cần thiết
    text = re.sub(r'[^a-z0-9\s-]', '', text)

    # Đổi khoảng trắng thành dấu gạch ngang
    text = re.sub(r'\s+', '-', text)

    # Xóa dấu - thừa
    text = text.strip('-')

    return text


base_url = "https://winmart.vn/info/danh-sach-cua-hang/"



df = pd.read_csv("store_info/province_list.csv")

df['slug'] = df['province'].apply(vietnamese_to_slug)
df['link'] = base_url + df['slug']

df.to_csv("store_info/links.csv", index=False)

df.head()


,province,slug,link
0,TP. Hà Nội,thanh-pho-ha-noi,https://winmart.vn/info/danh-sach-cua-hang/tha...
1,TP. Hồ Chí Minh,thanh-pho-ho-chi-minh,https://winmart.vn/info/danh-sach-cua-hang/tha...
2,TP. Đà Nẵng,thanh-pho-da-nang,https://winmart.vn/info/danh-sach-cua-hang/tha...
3,T. An Giang,tinh-an-giang,https://winmart.vn/info/danh-sach-cua-hang/tin...
4,T. Bắc Giang,tinh-bac-giang,https://winmart.vn/info/danh-sach-cua-hang/tin...


In [16]:
from selenium.common.exceptions import TimeoutException

def scroll_until_end(driver, pause_time=0.8, max_retry=50):
    last_height = driver.execute_script("return document.body.scrollHeight")
    retries = 0

    while retries < max_retry:
        # Scroll xuống thêm một đoạn
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        time.sleep(pause_time)  # Cho JS render thêm item

        # Chờ thêm shop mới xuất hiện
        try:
            WebDriverWait(driver, 2).until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "div.shop-liststyles__ShopCard-sc-w4fccm-2.jYpNtY")
                )
            )
        except TimeoutException:
            pass

        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            # Không tăng nữa → tăng counter retry
            retries += 1
        else:
            # Đã load thêm → reset retry
            retries = 0

        last_height = new_height

def scrape_shops_from_province(link):
    driver = webdriver.Chrome()
    driver.get(link)
    time.sleep(5)

    # # Lazy load scroll
    scroll_until_end(driver)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    shops_data = []

    # List các quận/huyện
    district_headers = soup.find_all("div", class_="shop-liststyles__TitleHeader-sc-w4fccm-4 kIDuvy")

    # List tất cả các shop card
    shop_cards = soup.find_all("div", class_="shop-liststyles__ShopCard-sc-w4fccm-2 jYpNtY")

    current_district = None
    shop_index = 0

    # Duyệt toàn bộ thẻ trên trang theo thứ tự xuất hiện
    for tag in soup.find_all(["div"]):

        # Update district name khi gặp TitleHeader
        if "shop-liststyles__TitleHeader-sc-w4fccm-4" in tag.get("class", []):
            current_district = tag.get_text(strip=True)

        # Khi gặp shop card thì lấy thông tin
        if "shop-liststyles__ShopCard-sc-w4fccm-2" in tag.get("class", []):
            name_tag = tag.find("div", class_="shop-liststyles__StoreHeader-sc-w4fccm-6")
            shop_name = name_tag.get_text(strip=True) if name_tag else None
            
            # thẻ address là thẻ <div> thứ 2
            addr_tags = tag.find_all("div")
            shop_addr = addr_tags[1].get_text(strip=True) if len(addr_tags) > 1 else None

            shops_data.append({
                "district": current_district,
                "shop_name": shop_name,
                "address": shop_addr
            })

            shop_index += 1

    return shops_data


In [43]:
hn_shops = scrape_shops_from_province('https://winmart.vn/info/danh-sach-cua-hang/thanh-pho-ha-noi')

result = pd.DataFrame(hn_shops)
result.to_csv("store_info/hn.csv", index=False)

In [ ]:
import pandas as pd 

province = pd.read_csv("store_info/links.csv")


for slug, link in zip(province['slug'], province['link']):
    shops = scrape_shops_from_province(link)

    result = pd.DataFrame(shops)
    result.to_csv(f'store_info/provinces/{slug}.csv', index=False)

